In [2]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
import random


Load in the Data

In [102]:
games_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/games.csv")
games_df

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23
...,...,...,...,...,...,...,...,...,...
131,2022110607,2022,9,11/06/2022,13:00:00,WAS,MIN,17,20
132,2022110608,2022,9,11/06/2022,16:05:00,ARI,SEA,21,31
133,2022110609,2022,9,11/06/2022,16:25:00,TB,LA,16,13
134,2022110610,2022,9,11/06/2022,20:20:00,KC,TEN,20,17


In [21]:
plays_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/plays.csv")
plays_df = plays_df[["quarter", "down", "yardsToGo", "possessionTeam","defensiveTeam","absoluteYardlineNumber", "gameClock", "preSnapHomeScore","preSnapVisitorScore","passResult", "offenseFormation", "defendersInTheBox"]]
#makes pass result binary(0 run, 1 pass)
plays_df['passResult'] = plays_df['passResult'].fillna(0).where(plays_df['passResult'].isna(),1)
#assigns each offesive formation a unique number classifier
offense_formations = plays_df['offenseFormation'].unique()
offense_formations_num = {value: i for i, value in enumerate(offense_formations)}
plays_df['offenseFormation'] = plays_df['offenseFormation'].map(offense_formations_num)
#assigns each possession team a unique number classifier
teams = plays_df.possessionTeam.unique()
teams_num = {value: i for i, value in enumerate(teams)}
plays_df['possessionTeam'] = plays_df['possessionTeam'].map(teams_num)
#assigns each defensive team a unique number classifier
plays_df['defensiveTeam'] = plays_df['defensiveTeam'].map(teams_num)
#make all game times purly numbers
plays_df['gameClock'] = plays_df['gameClock'].replace(':', "", regex =True).astype(int)

teams_num

{'ATL': 0,
 'PIT': 1,
 'LV': 2,
 'DEN': 3,
 'BUF': 4,
 'TEN': 5,
 'SF': 6,
 'KC': 7,
 'SEA': 8,
 'GB': 9,
 'NYJ': 10,
 'LA': 11,
 'JAX': 12,
 'DAL': 13,
 'DET': 14,
 'CAR': 15,
 'IND': 16,
 'MIN': 17,
 'CLE': 18,
 'WAS': 19,
 'CIN': 20,
 'BAL': 21,
 'PHI': 22,
 'TB': 23,
 'NO': 24,
 'ARI': 25,
 'NE': 26,
 'NYG': 27,
 'MIA': 28,
 'CHI': 29,
 'HOU': 30,
 'LAC': 31}

Seperating plays by team and creating a test and train dataset for each team(later we can do analysis on which teams we can predict the best)

In [24]:
teams = plays_df.possessionTeam.unique()
#empty dictionary of team names
plays_df_by_team_test = {x : pd.DataFrame() for x in teams}
plays_df_by_team_train = {x : pd.DataFrame() for x in teams}
plays_df_test = pd.DataFrame()
plays_df_train = pd.DataFrame()
#fills dictionary with all plays according to possesion team
for team in plays_df_by_team_test.keys():
    plays_df_by_team_test[team] = plays_df[:][plays_df.possessionTeam == team]
    #seperate into train and test sets seperated by team 
    rows = len(plays_df_by_team_test[team])
    rand_idx = np.random.randint(0, rows,size = int(rows/10))
    plays_df_by_team_train[team] = plays_df_by_team_test[team].drop(plays_df_by_team_test[team].index[rand_idx])
    plays_df_by_team_test[team] = plays_df_by_team_test[team].iloc[rand_idx]
    #create a joined train and test set
    plays_df_test = pd.concat([plays_df_test, plays_df_by_team_test[team]], ignore_index = True)
    plays_df_train = pd.concat([plays_df_train, plays_df_by_team_train[team]], ignore_index = True)
  
plays_df_test

,quarter,down,yardsToGo,possessionTeam,defensiveTeam,absoluteYardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,offenseFormation,defendersInTheBox
0,3,1,10,0,8,35,1500,20,17,0,0,7.0
1,2,1,10,0,24,92,347,10,7,1,4,8.0
2,4,1,10,0,31,68,1422,17,14,1,4,7.0
3,2,1,10,0,15,68,1254,0,3,1,4,7.0
4,2,1,10,0,6,40,702,14,14,0,4,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1228,2,1,10,31,18,20,633,14,10,0,1,8.0
1229,2,1,10,31,12,85,1352,0,6,0,0,6.0
1230,4,3,3,31,18,66,200,28,30,0,0,8.0
1231,4,1,10,31,18,74,1426,28,27,0,2,7.0


In [25]:
plays_test = np.array(plays_df_test)
plays_train = np.array(plays_df_train)

In [ ]:
tackles_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/tackles.csv")
tackles_df

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0
...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1
17422,2022092502,3510,42406,0,0,0,1
17423,2022091113,3642,43478,0,0,0,1
17424,2022091901,3578,42431,0,0,0,1
